In [1]:
import os
import torch
import random
import cv2.cv2
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision.transforms as transforms

from PIL import Image
from torch.autograd import Variable
from torch.optim.lr_scheduler import *
from torchvision.models.resnet import resnet18
from sklearn.model_selection import train_test_split

In [2]:
data_dir = 'E:\\code\\Python\\catanddog\\train'
test_dir = 'E:\\code\\Python\\catanddog\\test'

In [3]:
class Data(data.Dataset):
    def __init__(self, path, transform = None, train = True, test = False):
        self.test = test
        self.train = train
        self.transform = transform
        #imgs = [os.path.join(path, img) for img in os.listdir(path)]
        imgs = [os.path.join(data_dir, img) for img in path]
            
        if self.test:#test模式
            self.imgs = imgs
        else:#train模式
            random.shuffle(imgs)#数据打乱
            self.imgs = imgs
            
    def __getitem__(self, index):
        img = self.imgs[index]
        if self.test:
            label = 2
        else:
            label = 0 if 'cat' in img.split('\\')[-1] else 1#cat = 0, dog = 1
        image = Image.open(img)
        image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.imgs)

In [4]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.RandomCrop((224, 224)), 
    transforms.RandomHorizontalFlip(), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.2225)) 
])


transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
train_dirs, valid_dirs = train_test_split(os.listdir(data_dir), test_size = 0.2, random_state = 2021)
trainset = Data(train_dirs, transform = transform_train)
valset = Data(valid_dirs, transform = transform_val)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 20, shuffle = True, num_workers = 0)
valloader = torch.utils.data.DataLoader(valset, batch_size = 20, shuffle = False, num_workers = 0)

In [6]:
def get_acc(output, label):
    total = output.shape[0]
    _, pred_label = output.max(1)
    num_correct = (pred_label == label).sum().item()
    return num_correct / total

In [7]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    scheduler.step()
    model.train()
    train_acc = 0.0
    for batch_idx, (img, label) in enumerate(trainloader):
        image = Variable(img.cuda())
        label = Variable(label.cuda())
        optimizer.zero_grad()
        out = model(image)
        loss = criterion(out, label)
        loss.backward()
        optimizer.step()
        train_acc = get_acc(out, label)
        print("Epoch:%d [%d|%d] loss:%f acc:%f" % (epoch, batch_idx, len(trainloader), loss.mean(), train_acc))

In [8]:
def val(epoch):
    print("\nValidation Epoch: %d" % epoch)
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (img, label) in enumerate(valloader):
            image = Variable(img.cuda())
            label = Variable(label.cuda())
            out = model(image)

            _, predicted = torch.max(out.data, 1)

            total += image.size(0)
            correct += predicted.data.eq(label.data).cpu().sum()
    print("Acc: %f " % ((1.0 * correct.numpy()) / total))

In [9]:
class Net(nn.Module):
    def __init__(self, model):
        super(Net, self).__init__()
        # 取掉model的后1层
        self.resnet_layer = nn.Sequential(*list(model.children())[:-1])
        self.Linear_layer = nn.Linear(512, 2) #加上一层参数修改好的全连接层

    def forward(self, x):
        x = self.resnet_layer(x)
        x = x.view(x.size(0), -1)
        x = self.Linear_layer(x)
        return x

In [10]:
model = resnet18(pretrained=True)
model.fc = nn.Linear(512, 2,bias=True)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)  # 设置训练细节
scheduler = StepLR(optimizer, step_size=3)
criterion = nn.CrossEntropyLoss()

for epoch in range(1):
    train(epoch)
    val(epoch)
torch.save(model, 'catvsdog_model.pth')  # 保存模型


Epoch: 0


D:\Anaconda\envs\pytorch\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch:0 [0|320] loss:1.012006 acc:0.300000
Epoch:0 [1|320] loss:0.551931 acc:0.800000
Epoch:0 [2|320] loss:0.671688 acc:0.600000
Epoch:0 [3|320] loss:0.740110 acc:0.350000
Epoch:0 [4|320] loss:0.723512 acc:0.450000
Epoch:0 [5|320] loss:0.623624 acc:0.750000
Epoch:0 [6|320] loss:0.554391 acc:0.700000
Epoch:0 [7|320] loss:0.494984 acc:0.800000
Epoch:0 [8|320] loss:0.465446 acc:0.850000
Epoch:0 [9|320] loss:0.502569 acc:0.850000
Epoch:0 [10|320] loss:0.532694 acc:0.700000
Epoch:0 [11|320] loss:0.434402 acc:0.800000
Epoch:0 [12|320] loss:0.318724 acc:0.900000
Epoch:0 [13|320] loss:0.309133 acc:0.950000
Epoch:0 [14|320] loss:0.227396 acc:0.950000
Epoch:0 [15|320] loss:0.303179 acc:0.800000
Epoch:0 [16|320] loss:0.200223 acc:0.950000
Epoch:0 [17|320] loss:0.202507 acc:1.000000
Epoch:0 [18|320] loss:0.171531 acc:0.950000
Epoch:0 [19|320] loss:0.255217 acc:0.950000
Epoch:0 [20|320] loss:0.190764 acc:0.950000
Epoch:0 [21|320] loss:0.305590 acc:0.900000
Epoch:0 [22|320] loss:0.304660 acc:0.85000

In [11]:
classes = ['cat', 'dog']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load('catvsdog_model.pth')  # 加载模型
model = model.to(device)
model.eval()  # 把模型转为test模式

img = cv2.imread("E:\\code\\Python\\catanddog\\test\\dog.3886.jpg")  # 读取要预测的图片
cv2.imshow("img", img)
img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
trans = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

img = trans(img)
img = img.to(device)
img = img.unsqueeze(0)  # 图片扩展多一维,因为输入到保存的模型中是4维的[batch_size,通道,长，宽]，而普通图片只有三维，[通道,长，宽]
output = model(img)
prob = F.softmax(output, dim=1)  # prob是2个分类的概率
value, predicted = torch.max(output.data, 1)
pred_class = classes[predicted.item()]
print(pred_class)

dog
